In [1]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import numpy.typing as npt
import cProfile

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 



In [6]:
class Biome():
    def __init__(self, grid: npt.NDArray[np.int8]):
        self.grid = grid
        self.val_grid = np.zeros(grid.shape, dtype=np.int8)

    def __repr__(self):
        return str(self.grid)
    
    def run(self, generations: int):
        for i in range(generations):
            self.grid = self._update(self.grid)
        
    def _update(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        
        if np.sum(grid) == 0:
            return grid

        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]

        # get the unique surrounding indices
        each_indices = np.unique(surrounding_indices, axis=0)

        # set the surrounding indices to 1 if they have 3 neighbours
        grid[each_indices[:, 0], each_indices[:, 1]] = [1 if np.sum(np.all(surrounding_indices == i, axis=1)) == 3 else 0 for i in each_indices]
        grid[indices[:, 0], indices[:, 1]] = [1 if np.sum(np.all(surrounding_indices == i, axis=1)) == 2 else 0 for i in np.unique(indices, axis=0)]
        
        return grid
    
    def values(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]
        
        val_grid = grid.copy()

        for i in np.unique(surrounding_indices, axis=0):
            val_grid[i[0], i[1]] = np.sum(np.all(surrounding_indices == i, axis=1))
        
        return val_grid

In [13]:
seed = np.random.randint(0, 1, size=(50, 50), dtype=np.int8)

grid = np.zeros((1000, 1000), dtype=np.int8)
grid[500:550, 500:550] = seed

biome = Biome(grid)
cProfile.run('biome.run(10000)')

         100004 function calls in 8.447 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.010    0.000    8.436    0.001 2676004308.py:13(_update)
        1    0.011    0.011    8.447    8.447 2676004308.py:9(run)
    10000    0.008    0.000    8.427    0.001 <__array_function__ internals>:177(sum)
        1    0.000    0.000    8.447    8.447 <string>:1(<module>)
    10000    0.001    0.000    0.001    0.000 fromnumeric.py:2157(_sum_dispatcher)
    10000    0.017    0.000    8.410    0.001 fromnumeric.py:2162(sum)
    10000    0.020    0.000    8.390    0.001 fromnumeric.py:69(_wrapreduction)
    10000    0.006    0.000    0.006    0.000 fromnumeric.py:70(<dictcomp>)
        1    0.000    0.000    8.447    8.447 {built-in method builtins.exec}
    10000    0.003    0.000    0.003    0.000 {built-in method builtins.isinstance}
    10000    0.007    0.000    8.417    0.001 {built-in method numpy.core._multiarray